In [1]:
import json
from datetime import datetime

import pandas as pd
import requests
from sqlalchemy import select
from sqlalchemy.orm import aliased

from database import Database
from utils import DistributorApi

In [2]:
api = DistributorApi()
db = Database()

In [18]:
# stop sales rules from Distributor API

rules = api.get_all_rules()
df_rules = pd.json_normalize(rules, sep="_")
df_rules = df_rules[
    [
        "id",
        "name",
        "description",
        "tag",
        "edit_state",
        "obsolete",
        "updated_by",
        "updated_on",
        "credential_level",
        "credential_list",
        "provider_level",
        "provider_list",
        "hotel_level",
        "hotel_list",
        "destination_level",
        "destination_list",
        "rate",
        "refundable",
        "market_level",
        "market_list",
        "meal_level",
        "meal_list",
        "check_in_level",
        "check_in_from",
        "check_in_to",
        "booking_date_level",
        "booking_date_from",
        "booking_date_to",
        "range_level",
        "range_from",
        "range_to",
        "max_release",
        "days_of_week_level",
        "days_of_week_list",
        "age",
        "room_level",
        "room_list",
        "num_of_nights_level",
        "num_of_nights_list",
        "hours_level",
        "hours_list",
        "dynamic_commission",
    ]
]

In [19]:
df_rules.shape

(3327, 42)

In [20]:
# clients_credential from discovery

cc = aliased(db.clients_credential)
df_credentials = pd.read_sql(
    sql=select(cc.c.id.label("credential_id"), cc.c.enabled), con=db.engine.connect()
)

In [21]:
# fitler for tags empty, product, customer request and credential list

df_rules_filtered = df_rules[
    (df_rules["credential_level"] == 1) & (df_rules["tag"].isin([-1, 1, 3]))
] 
df_rules_expanded = df_rules_filtered.explode("credential_list").reset_index(drop=True)

In [22]:
df_rules_filtered.shape

(1116, 42)

In [23]:
df_rules_expanded["credential_id"] = df_rules_expanded["credential_list"]
df_rules_expanded = df_rules_expanded.merge(
    df_credentials, on="credential_id", how="left"
)

In [24]:
df_rules_expanded.shape

(141752, 44)

In [25]:
df_regrouped = (
    df_rules_expanded.groupby(["id"])
    .agg(
        credential_count=("credential_id", "count"),
        enabled=("enabled", lambda x: (x == True).sum()),
        disabled=("enabled", lambda x: (x == False).sum()),
    )
    .reset_index()
)

df_regrouped["disabled_percentage"] = (df_regrouped["disabled"] * 100.0) / df_regrouped[
    "credential_count"
]

# filter for rules where 100% of the credentials are disabled
# df_regrouped = df_regrouped[df_regrouped["disabled_percentage"] == 100]

In [26]:
df_regrouped.shape

(1116, 5)

In [27]:
df_rules_inactive_creds = df_rules_filtered.merge(df_regrouped, on="id", how="right")

In [28]:
df_rules_inactive_creds.shape

(1116, 46)

In [29]:
df_rules_inactive_creds.to_csv("df_rules_inactive_creds.csv", index=False)

In [30]:
# rule_ids = list(df_regrouped["id"])
# rule_ids

In [31]:
# Uncomment to delete
# deleted_rules = api.delete_rules(rule_ids)

In [32]:
df_rules_inactive_creds.to_csv(
    f"deleted_rules_{datetime.today().strftime('%Y%m%d_%H%M%S')}_with_inactive_creds.csv",
    index=False,
)

In [34]:
df_rules_expanded[df_rules_expanded["enabled"]].shape

(83364, 44)